In [1]:
import json
import numpy as np
import wowsims

This example loads simulation settings for a feral druid.
These are stored in a json file that is included in the repository.

In [2]:
f = open('data/feral.json')
settings = json.load(f)

The sim can be run with these settings just like you would on the web app.

In [3]:
result = wowsims.runSim(settings)

The `result` is a json object that corresponds to the raid sim result.

In [7]:
result.keys(), result['raidMetrics']['dps']['avg']

(dict_keys(['raidMetrics', 'encounterMetrics', 'logs', 'firstIterationDuration', 'avgIterationDuration']),
 10512.447776301875)

The bindings for the sim also allow for finer-grained control. Specifically, you can step through a single iteration.
Before stepping through the sim, it needs to be reset from the previous run.
The easiest way to reset a simulation, for now, is to create a new one.
To do that, the settings json is encoded as a utf-8 string and passed as input to `new`.

In [8]:
def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))

The sim library currently ignores the iteration parameter in the settings to better support interactive mode (see a few cells down).
The following code is an example of how to run multiple iterations in this environment.

In [9]:
iterations = settings['simOptions']['iterations']
duration = settings['encounter']['duration']

In [ ]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Interactive mode is a way to provide input to the sim while it is running.
This can be used, for example, as a way to test rotation ideas.
Because interactive mode is enabled, the code below does nothing other than auto attack.

In [ ]:
settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

The next run uses the shred spell when it can.
Spells are accesed via the casting character's spellbook.
In this case, we have a help file `feral.py` that defines some globals for the spells we are interested in.

In [ ]:
from feral import Spells

Spells.register()

settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            wowsims.trySpell(Spells.Shred)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Cooldowns can be queried simialr to auras. The `feral.py` file has some code to automatically track cooldowns for registered actions.

In [ ]:
settings['simOptions']['interactive'] = False
reset()

while not wowsims.step():
    casted_berserk = wowsims.trySpell(Spells.Berserk)
    if casted_berserk:
        Spells.update()
        print(Spells.Cooldowns)


For more complex input logic having some context would be helpful.
The `feral.py` file also contains some aura labels and registers them with the sim. 

In [ ]:
from feral import Auras, TargetAuras

Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            Auras.update()
            TargetAuras.update()
            eng = wowsims.getEnergy()
            cp = wowsims.getComboPoints()
            if cp > 0 and Auras.get_dur('Savage Roar Aura') <= 0.5:
                wowsims.trySpell(Spells.Roar)
            elif cp < 5:
                wowsims.trySpell(Spells.Shred)
            else:
                wowsims.trySpell(Spells.Bite)

    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

The examples so far have only looked at damage output.
More detailed spell metrics are also available.

In [ ]:
settings['simOptions']['interactive'] = False

reset()
while not wowsims.step():
    pass
cast_metrics = wowsims.getSpellMetrics()
for spell_id, metrics in cast_metrics.items():
    print(spell_id, [metric['Casts'] for metric in metrics])